In [10]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from math import *

%matplotlib inline

In [30]:
def backtest(data):
    len_time = len(data.rows)
    
    slippage = 0.0005
    leverage = 2
    win_len = 60
    
    PNL = np.zeros(len_time-60)
    PNL[0] = 100
    
    positions = np.zeros(len(pairs))
    new_positions = np.zeros(len(pairs))
    beta_array = np.zeros(len(pairs))
    
    
    trading_pairs = [pairs, positions]    
    for i in range(len(pairs)):
        A_data = data[data['COMNAM']==pairs[i][0]]
        A_price = A_data['PRC'][0:win_len-1]
        B_data = data[data['COMNAM']==pairs[i][1]]
        B_price = B_data['PRC'][0:win_len-1]
             
        signal = generate_signal(A_price, B_price)
        beta[i] = signal[0]
        signal = signal[1]
            
        if signal == 'BUYOPEN':
            positions[i] = 1
        elif signal == 'SELLOPEN':
            positions[i] = -1
        else:
            positions[i] = 0
            
    invest_per_pair = PNL[0] * leverage / sum(positions)
    positions = positions * invest_per_pair
        
    for t in range(win_len + 1,len_time): 
        PNL_part = 0
        for i in range(len(pairs)):
            A_data = data[data['COMNAM']==pairs[i][0]]
            A_bid = A_data['BID'][0:win_len]
            A_ask = A_data['ASK'][0:win_len]
            A_price = A_data['PRC'][0:win_len-1]
            B_data = data[data['COMNAM']==pairs[i][1]]
            B_bid = B_data['BID'][0:win_len]
            B_ask = B_data['ASK'][0:win_len]
            B_price = B_data['PRC'][0:win_len-1]
            
            A_prev_bid = A_bid[-2]
            A_prev_ask = A_ask[-2]
            B_prev_bid = B_bid[-2]
            B_prev_ask = B_ask[-2]
            A_current_bid = A_bid[-1]
            A_current_ask = A_ask[-1]
            B_current_bid = B_bid[-1]
            B_current_ask = B_ask[-1]
            
            if position[i] > 0:
                A_shares = -positions[i]/(A_prev_bid - beta[i]*B_prev_ask)
                B_shares = -beta[i] * A_shares
                PNL_part += A_shares * (A_current_bid - A_prev_bid) + B_shares * (B_current_ask - B_prev_ask)
            elif position[i] < 0:
                A_shares = -positions[i]/(A_prev_ask - beta[i]*B_prev_bid)
                B_shares = -beta[i] * A_share
                PNL_part += A_shares * (A_current_ask - A_prev_ask) + B_shares * (B_current_bid - B_prev_bid)
            
            signal = generate_signal(A_price, B_price)
            beta[i] = signal[0]
            signal = signal[1]
            
            if signal == 'BUYOPEN':
                new_positions[i] = 1
            elif signal == 'SELLOPEN':
                new_positions[i] = -1
            else:
                new_positions[i] = 0
        
        invest_per_pair = PNL_array[0] * leverage / sum(new_positions)
        new_positions = new_positions * invest_per_pair
        PNL[t-60] = PNL_part + PNL[t-61] * (1 + r) - sum(positions) * r - sum(abs(new_positions - positions)) * slippage
        positions = new_positions 
        
    return PNL
            